In [6]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import os
import sys
import urllib.request
import requests
import time
import nltk

import googletrans
from sklearn.model_selection import train_test_split

## 데이터 불러오기

In [7]:
data=pd.read_csv('./data/Lyrics.csv')
data

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0
1,라라랄라 라라랄라 워어 워어 너가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 ...,0
2,자꾸 마주치는 네 눈빛에 Baby baby왠지 어색해 대체 왜 이래고개를 숙여봐도너...,0
3,어깨동무까진 괜찮아그치만 손 잡는 건 조금 위험해잘 자란 메세진 괜찮아그래도 자냐는...,0
4,햇살이 가득그댈 향한 가벼운 발걸음콧노래 부르며쇼윈도에 비친내 모습을 한번 점검해시...,0
...,...,...
741,\r\n\t\t\t\t\t\t\t반쯤 미쳐가는 나망가져 나의 삶넌 내게서 나를 지워...,3
742,\r\n\t\t\t\t\t\t\t무궁화 꽃이 피었습니다우리 빼고 모두 다 멈춰지담이...,3
743,\r\n\t\t\t\t\t\t\t아예 아예 아예 아예 아예 아예 아예 아예 아예 아...,3
744,\r\n\t\t\t\t\t\t\t난 나쁜 기집애 나난 나쁜 기집애 난 나쁜 기집애...,3


## 구글번역

In [8]:
def checkKorean(str):
    pattern = '[ㄱ-ㅎ가-힣\s]+'
    if re.match(pattern, str):
        return True
    else:
        return False

In [9]:
translator=googletrans.Translator()
trans=[]
for i,lyric in enumerate(data['Lyrics']):
    split_sentences = re.compile("[ㄱ-ㅎ가-힣\s]+|[a-zA-Z\s']+").findall(lyric)
    sen=''
    for sentence in split_sentences:
        check=checkKorean(sentence)
        if not check:
            try:
                en2ko=translator.translate(sentence,dest='ko',src='en')
                sen+=en2ko.text
            except:
                sen+=sentence
        else:
            sen+=sentence
    trans.append(sen)
len(trans)

746

In [10]:
lyrics=[]
for lyric in trans:
    sub=re.sub("[0-9a-zA-Z'.?!()&^%$#,\t\r\n]+",' ',lyric)
    sub=sub.strip()
    lyrics.append(re.sub(' +', ' ', sub))
lyrics[:2]

['아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하고 있어 늦은 밤 졸린 눈 비비며 너에게 편지를 쓸게 반가운 사진 속 그 표정만큼 아름다운 나날들이기를 찬 바람이 불어와 어깨가 움추러들 때에도 너를 떠올린다면 씩씩한 표정 할수있어 알고있니 넌 나의 숫자 두근대는 맘은 아직까지 여기에 언제까지나 나의 하나 뿐인좋아한단 말은 나를 웃음짓게 해 내 번호 나의 하나 뿐인때로는 지쳐 울어도 내 번호 나는 당신과 함께 있고 싶습니다우 우 우 우내 번호 눈을 감으니 꼭 곁에 있는것 같아쨘 하고 나타날게생각치 못한 어느 날우연히 문득 뒤돌아 봤을 때그때 꼭 내가 서 있을게꼭 하고 껴안을게 가장 반가운 얼굴로눈물이 왈칵 쏟아진다해도나는 환하게 웃을거야혹시 오늘 너에게서러운 일이 생긴다면오래 전 그 날처럼 달려가꼭 안아주고파아름답게 빛나길알고있니 넌 나의 숫자두근대는 맘은 아직까지 여기에언제까지나 나의 하나 뿐인좋아한단 말은 나를 웃음짓게 해내 번호 나의 하나 뿐인때로는 지쳐 울어도내 번호난 난 여전히 당신과 사랑에 빠지고 싶어 내 번호눈을 감으니 꼭 곁에 있는것 같아 기억해줘 넌 나의 숫자보고싶은 사람 이만 줄일게 안녕',
 '라라랄라 라라랄라 워어 워어 너가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 싶어 너가 좋아했던 것들을 나에게 자그마한 행복들 달력 안 기념일들로 채워지네 채워지네 응나나나나나나 너가 좋아했던 살구빛 샐러드 그 날은 샐러드 기념일 우후나나나나나 너가 좋아하던 멜로디언 소리 그 날은 멜로디 기념일 우 라라라나나나 너가 예뻤었다고 했던 조그만 고양이 귀여워서 또 보고 싶어 너가 좋아했던 것들을 나에게 자그마한 고양이들 방에 한가득하게 채워지네 채워지네 응나나나나나나 너가 좋아했던 살구빛 샐러드 그 날은 샐러드 기념일 우후 나나나나 너가 좋아하던 멜로디언 소리 그 날은 멜로디 기념일 우우 라라라나나나 너가 좋아했던 살구빛 샐러드 그 날은 샐러드 기념일 우후 나나나나 너가 좋아하던 멜로디언 소리 그 날은 멜로디 기념일

In [11]:
len(lyrics)

746

In [12]:
data['Lyrics']=lyrics
data

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0
1,라라랄라 라라랄라 워어 워어 너가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...,0
2,자꾸 마주치는 네 눈빛에 아기 아기왠지 어색해 대체 왜 이래고개를 숙여봐도너의 발끝...,0
3,어깨동무까진 괜찮아그치만 손 잡는 건 조금 위험해잘 자란 메세진 괜찮아그래도 자냐는...,0
4,햇살이 가득그댈 향한 가벼운 발걸음콧노래 부르며쇼윈도에 비친내 모습을 한번 점검해시...,0
...,...,...
741,반쯤 미쳐가는 나망가져 나의 삶넌 내게서 나를 지워가고또 나를 부셔버려라추락한 나를...,3
742,무궁화 꽃이 피었습니다우리 빼고 모두 다 멈춰지담이 꽃이 피었습니다현아 빼고 모두 ...,3
743,아예 아예 아예 아예 아예 아예 아예 아예 아예 아예 왜 이렇게 감을 못 잡는거야 ...,3
744,난 나쁜 기집애 나난 나쁜 기집애 난 나쁜 기집애 내 모든 나쁜 여자가있는 곳난 나...,3


In [14]:
data[data['Lyrics']=='']

,Lyrics,label


In [15]:
data.to_csv('./data/Lyrics_trans.csv',encoding='utf-8-sig',index=False)